In [1]:
import os 
import sys 
import glob 
import numpy as np 
from math import *
from pointCloudsDescriptor import PointCloudFEvaluator

%matplotlib inline 
import matplotlib.pyplot as plt 

# Methods definition - plot histograms

In [ ]:
def plot_comparison(datas, nbbins = 80):
    """
    Estimate and compare the histograms of different point clouds 
    NOTE: Like in the process all the densities are mixed, the scale of the 
    frequencies wont be 1
    :Arguments:
        :datas: dict, {'histName0': list(densities0), ...., "histNameN": list(densitiesN) }
        :nbbins: int, number of bins to calculate 
    """
    _, globalbins = np.histogram(np.concatenate(list(datas.values())), bins=nbbins)
    plt.figure(figsize=(15,5))
    for name, idensities in datas.items():
        hist, bins = np.histogram(idensities, bins=globalbins)
        hist = hist/len(idensities)
        bins = bins / 10**6
        width = np.diff(bins)
        binscenters = (bins[:-1] + bins[1:]) / 2 
        plt.bar(binscenters, hist, align='center', width=width, alpha=0.5)
        plt.plot(bins[:-1]+width/2, hist, label = name)
    plt.legend()

In [ ]:
def compare_densities(dicDensities, nbins=80):
    maxDensities_freq = 0
    _, bins_of_ref = np.histogram(np.concatenate(list(dicDensities.values())), bins=nbins)
    legends = []
    maxFreqPc = ""
    # Select the frequencie of reference 
    for a_key in dicDensities.keys():
        a_freq, _ = np.histogram(dicDensities[a_key], bins=bins_of_ref)
        if(max(a_freq) > maxDensities_freq):
            maxDensities_freq = max(a_freq)
            maxFreqPc = a_key
        else:
            continue 
    print("The maximu freq correspond to %s" %(maxFreqPc))
            
    for a_key in dicDensities.keys():
        a_freq, _ = np.histogram(dicDensities[a_key], bins=bins_of_ref)
        # Scale the frequencies
        a_freq = a_freq/maxDensities_freq
        # Bar parameters
        binscenters = (bins_of_ref[:-1] + bins_of_ref[1:]) / 2 
        width = (bins_of_ref[1]-bins_of_ref[0])
        #Plot
        plt.bar(binscenters/max(bins_of_ref), a_freq, width/max(bins_of_ref), align='center')
        legends.append(a_key)
        plt.legend(legends)
        

# Load pointclouds

In [ ]:
def load_datas(path, files_format="txt"):
    """
    Create a dictionary with the numpy arrays that represent each 
    point cloud 
    
    :Arguments:
        :path: str, Path to the folder with the point clouds
        :files_format: str, String with the file format, default: txt
    :Output:
        dict, {'file_name0':np.array[] .... 'file_nameN':np.array[]}
    """
    files = glob.glob( os.path.join( path, "*.%s" %(files_format)) )
    print("-> Found %i files\n" %(len(files)))
    datas = {}
    for  idx1, afile in enumerate(files):
        pointcloud_name = os.path.split(afile)[-1].split(".")[0]
        print("  -> Load file [%i/%i]" %(idx1+1, len(files)), end="\r" if (idx1<len(files)) else "\n" )
        try:
            datas[pointcloud_name] = np.loadtxt(afile)[:,0:3]
        except ValueError as err:
            datas[pointcloud_name] = np.loadtxt(afile, delimiter=",")[:,0:3]
    return datas

# Calculate the point cloud densities -- Density per point 

In [ ]:
def compute_densities(datas, radius=0.1):
    pcEval = PointCloudFEvaluator()
    densities = {}
    for  idx1, (afile,a_pc) in enumerate(datas.items()):
        print("Density file [%i/%i]" %(idx1+1, len(datas)), end="\r" if (idx1<len(datas)) else "\n" )
        densities[afile] = pcEval.get_point_cloud_density(a_pc, radius)
    return densities

In [ ]:
def compute_densities2(path, radius=0.1):
    print(" -> Loading Data")
    real_data_pointclouds = load_datas(path)
    print(" -> Calculating Densities")
    real_scans_densities = compute_densities(real_data_pointclouds, radius=radius)
    print("-> OK")
    return real_scans_densities 

# Start Processing 

In [ ]:
real_data_path =  "/home/jprb/Documents/annotated_real_data/noFloor/"
synthetic_sres006 = "/home/jprb/Documents/synthetic_exp/220725_new_virtual_scan/to_evaluate/sress_006/"

In [ ]:
real_scans_densities = compute_densities2(real_data_path)

In [ ]:
densities_synthetic = compute_densities2(synthetic_sres006)

In [ ]:
dict2plot =  {
                "real densities" : np.concatenate(list(real_scans_densities.values())),
                "fake_densities" : np.concatenate(list(densities_synthetic.values())),
              }

In [ ]:
compare_densities(dict2plot, nbins=100)